1. Знайти кількість злочинів, що сталися у грудні 2012 року в інтервалі між першою годиною ночі та п'ятою ранку (`CrimesData`)

In [2]:
import pandas as pd
import numpy as np

In [3]:
def fix_time(cell):
    if len(cell) == 4 and ':' not in cell:
        return f'{cell[:2]}:{cell[2:]}:00'
    else:
        return cell

crimes_data = pd.read_excel(
    'Заняття6/Files/CrimesData.xlsx',
    sheet_name='2012',
)

crimes_data['CrimeTime'] = crimes_data['CrimeTime'].apply(fix_time)

crimes_data['CrimeDateTime'] = crimes_data['CrimeDate'] + ' ' + crimes_data['CrimeTime']
crimes_data.drop(['CrimeDate', 'CrimeTime'], axis=1, inplace=True)
crimes_data['CrimeDateTime'] = pd.to_datetime(
    crimes_data['CrimeDateTime'],
    format='%m/%d/%Y %H:%M:%S'
)

len(crimes_data[
    (crimes_data.CrimeDateTime.dt.month == 12) & 
    (crimes_data.CrimeDateTime.dt.time >= pd.to_datetime('01:00:00').time()) & 
    (crimes_data.CrimeDateTime.dt.time <= pd.to_datetime('05:00:00').time())
])

41

2. Знайти середню, мінімальну і максимальну вартість квитка в залежності від місця посадки і класу каюти (`Титанік`)

In [89]:
titanic_data = pd.read_csv('Заняття6/Files/TitanicDataset.csv', sep=',', header=0)

agr_data = titanic_data.groupby(by=['Embarked', 'Pclass']).agg(
    min_fare=('Fare', np.min),
    max_fare=('Fare', np.max),
    mean_fare=('Fare', np.mean),
)
agr_data.head(9)

min_fare  max_fare   mean_fare
Embarked Pclass                                
C        1        26.5500  512.3292  104.718529
         2        12.0000   41.5792   25.358335
         3         4.0125   22.3583   11.214083
Q        1        90.0000   90.0000   90.000000
         2        12.3500   12.3500   12.350000
         3         6.7500   29.1250   11.183393
S        1         0.0000  263.0000   70.364862
         2         0.0000   73.5000   20.327439
         3         0.0000   69.5500   14.644083

3. Найти загальну суму продажу по місяцях 2019-го року; вивести в форматі місяця - сума в порядку збільшення суми (`Sales`)

In [90]:
sales_data = pd.read_csv('Заняття6/Files/SalesData.csv', sep=',', header=0)
sales_data['OrderDate'] = pd.to_datetime(sales_data['OrderDate'])

# sales_data['OrderMonth'] = sales_data['OrderDate'].dt.strftime("%B")

sales_data[sales_data['OrderDate'].dt.year == 2019].groupby(
    by=[sales_data['OrderDate'].dt.strftime("%B")]).agg(
    MonthTotal=('Total', np.sum)).sort_values(
    by=['MonthTotal'], ascending=True, inplace=False)

,MonthTotal
OrderDate,
March,167.44
August,174.65
April,448.65
September,505.84
May,512.78
February,719.37
November,778.89
October,827.08
June,988.50


4. Знайти всіх виконавців, у яких понад 10 альбомів (`chinook`/`albums`)

In [75]:
import sqlite3

connection = sqlite3.connect('Заняття6/Files/chinook.db')

chinook_data = pd.read_sql_query(
    sql="SELECT * FROM albums", 
    con=connection 
)
album_count = chinook_data.groupby(by=['ArtistId']).agg(
    count_alb = ('AlbumId', 'count')
)
album_count[album_count['count_alb'] > 10]

,count_alb
ArtistId,
22,14
58,11
90,21


In [78]:
# chinook_data_sql = pd.read_sql_query(
#     sql="SELECT ArtistId, count(AlbumId) FROM albums GROUP BY ArtistId HAVING count(AlbumId)>10", 
#     con=connection 
# )
# print(chinook_data_sql)

   ArtistId  count(AlbumId)
0        22              14
1        58              11
2        90              21


5. Знайти всіх родичів по прізвищу. В результаті повинна вийти таблиця з двома колонками. У першій колонці повне ім&#39;я пасажира, у другій його родичів на борту (вважати родичами пасажирів, у яких збігається прізвище) (`Titanic`)

In [ ]:
titanic_data = pd.read_csv('Заняття6/Files/TitanicDataset.csv', sep=',', header=0)
titanic_data['LastName'] = titanic_data['Name'].apply(
    lambda x: x.split(',')[0])

titanic_relatives = pd.merge(
    left = titanic_data[['LastName', 'Name']],
    right = titanic_data[['LastName', 'Name']],
    on = "LastName",
    how = 'inner')
titanic_relatives.drop(['LastName'], axis=1, inplace=True)
titanic_relatives.rename(
    columns = {'Name_x': 'Name', 'Name_y': 'Relatives'}, inplace = True)
titanic_relatives[titanic_relatives['Name'] != titanic_relatives['Relatives']].groupby('Name').agg(
    {'Relatives': lambda x: '; '.join(x)}
)